# PydanticOutputParser

In [92]:
!pip install --upgrade --quiet langchain langchain-openai

In [28]:
import os
os.environ["OPENAI_API_KEY"]      = "sk-*****************************************************"

In [113]:
model = ChatOpenAI(temperature=0)

In [112]:
from typing import List

from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import ChatOpenAI

### example 1

In [114]:
# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

    # You can add custom validation logic easily with Pydantic.
    @validator("setup")
    def question_ends_with_question_mark(cls, field):
        if field[-1] != "?":
            raise ValueError("Badly formed question!")
        return field


# And a query intented to prompt a language model to populate the data structure.
joke_query = "Tell me a joke."

# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

chain.invoke({"query": joke_query})

Joke(setup="Why couldn't the bicycle stand up by itself?", punchline='Because it was two tired!')

### example 2

In [115]:
# Here's another example, but with a compound typed field.
class Actor(BaseModel):
    name: str = Field(description="name of an actor")
    film_names: List[str] = Field(description="list of names of films they starred in")


actor_query = "Generate the filmography for a random actor."

parser = PydanticOutputParser(pydantic_object=Actor)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

chain.invoke({"query": actor_query})

Actor(name='Tom Hanks', film_names=['Forrest Gump', 'Cast Away', 'Saving Private Ryan', 'Toy Story', 'The Green Mile', 'Apollo 13'])

### example 3

In [124]:
email_contents = """
From: jiwon Shin (dunndunn@dunndunn.com)
To: duck Kim (duck@duck.com)
Subject: "AI engineers and Robots are getting real

Welcome back, AI enthusiasts!
This week was simply mind-blowing, with two insane demos dropping that had our jaws on the floor - a coding AI that can build full apps autonomously, and a super realistic robot that understands and acts just like us.
In today’s email:
•	Meet Devin - The World's First Autonomous AI Coding Engineer
•	Figure Unleashes Jaw-Dropping AI Robot in OpenAI Collab
•	Claude 3 Pro or ChatGPT Plus: Which subscription is better?



News
Meet Devin - The World's First Autonomous AI Coding Engineer
The startup Cognition has launched a groundbreaking new tool called Devin that is pushing the boundaries of AI-powered software development and taking the internet by storm.
This isn't the first time we've seen language models that can write code - powerful tools like GitHub Copilot already exist. But what makes Devin truly special is its ability to autonomously build entire products from start to finish.
 	Cognition @cognition_labs


Today we're excited to introduce Devin, the first AI software engineer.
Devin is the new state-of-the-art on the SWE-Bench coding benchmark, has successfully passed practical engineering interviews from leading AI companies, and has even completed real jobs on Upwork.
Devin is… twitter.com/i/web/status/1…



Mar 12, 2024



42.7K Likes   10.1K Retweets   3.97K Replies


How Does Devin Work?
Devin is essentially a combination of multiple AI agents working together to perform the tasks of a human programmer. It comprises an integrated development environment (IDE), a terminal to run and debug code, and a browser to search for documentation online and troubleshoot issues.
Previous attempts at building such coding AI agents suffered from major limitations, frequent crashes, and bugs. It appears the team at Cognition has cracked the code on an underlying framework that seamlessly connects these various agents - succeeding where big tech companies have struggled.
The exact model powering Devin remains undisclosed, leading to speculation that it could be GPT-4 or a customized fine-tuned version of another large language model.
In the SWE-Bench benchmark, where an AI must autonomously solve real coding problems from open-source GitHub projects, Devin achieved an unprecedented score of 13.86%. For context, the previous high score was just 1.98% - a massive leap in capabilities.


What Can Devin Do?
👉 Fully complete coding jobs found on Upwork

👉 Pass coding interviews at leading tech companies
👉 Fine-tune other language models

👉 Develop a browser extension

👉 Build simple games
It's important to note that for now, Devin is in a limited beta with only a few developers having access. Once it opens up more broadly, we'll get to see many more real-world use cases and better understand the true impact.
In my opinion, we still have a long road ahead before language models can entirely replace human programmers at tech companies. But Devin represents a breakthrough not just in AI-powered coding, but in the broader field of AI agents collaborating on complex tasks.
This is definitely a development worth following closely as AI pushes further into the realm of software engineering. The era of autonomous AI coding may be closer than we think.
"""

class EmailSummary(BaseModel):
     person: str = Field(description="Person who sent the email")
     email: str = Field(description="Email address of sender")
     subject: str = Field(description="Email subject")
     summary: str = Field(description="Text summarizing the body of the email")
     date: str = Field(description="Meeting date and time mentioned in the email body")

parser = PydanticOutputParser(pydantic_object=EmailSummary)

prompt = PromptTemplate.from_template(
    """
You are a helpful assistant. Please answer the following questions in KOREAN.

QUESTION:
{question}

EMAIL CONTENTS:
{email_contents}

FORMAT:
{format}
"""
)

prompt = prompt.partial(format=parser.get_format_instructions())

chain = prompt | llm

response = chain.invoke(
    {
        "email_contents": email_contents,
        "question": "이메일 내용중 주요 내용을 추출해 주세요.",
    }
)

print(response.content)

{
  "person": "jiwon Shin",
  "email": "dunndunn@dunndunn.com",
  "subject": "AI engineers and Robots are getting real",
  "summary": "This week was mind-blowing with two insane demos - a coding AI that can build full apps autonomously and a super realistic robot that acts like us. Meet Devin - The World's First Autonomous AI Coding Engineer and Figure Unleashes Jaw-Dropping AI Robot in OpenAI Collab. Devin is a combination of multiple AI agents working together to perform tasks of a human programmer. It has achieved an unprecedented score of 13.86% in the SWE-Bench benchmark. Devin can complete coding jobs on Upwork, pass coding interviews at leading tech companies, fine-tune other language models, develop a browser extension, and build simple games. Devin is in a limited beta currently. The era of autonomous AI coding may be closer than we think.",
  "date": "Mar 12, 2024"
}
